# Imports

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
import joblib
import optuna
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns

# Settings

In [4]:
# Exports
LOCAL_EXPORT_FOLDER_PATH='/content/exports'
# Exports > Manual check path 
LOCAL_EXPORT_MANUAL_CHECK_PATCH_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/manual_check_patch'
TARGET_COLUMNS=['TARGET',]
LOCAL_EXPORT_MODELIZATION_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/modelization'
MLFLOW_EXPERIMENT_NAME = 'generic_model_experiment'
# Export > General Settings
TESTING_MODE=True
TESTING_MODE_MAX_LINES=1000
TESTING_MODE_SUB_FOLDER_NAME='testing_data'
GENERAL_CHUNK_SIZE=100000

In [1]:


# Liste des modèles et leurs hyperparamètres
models = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': np.logspace(-3, 3, 7),
            'solver': ['newton-cg', 'lbfgs', 'liblinear']
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 4, 5],
            'subsample': [0.8, 0.9, 1.0]
        }
    },
    'XGBoost': {
        'model': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 4, 5],
            'colsample_bytree': [0.3, 0.7]
        }
    },
    'LightGBM': {
        'model': lgb.LGBMClassifier(),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [31, 62, 127],
            'boosting_type': ['gbdt', 'dart']
        }
    },
    'CatBoost': {
        'model': cb.CatBoostClassifier(verbose=0),
        'params': {
            'iterations': [100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1],
            'depth': [3, 4, 5],
            'l2_leaf_reg': [3, 5, 7]
        }
    },
    'SVM': {
        'model': SVC(probability=True),
        'params': {
            'C': np.logspace(-3, 3, 7),
            'kernel': ['linear', 'rbf', 'poly'],
            'degree': [3, 4, 5]
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [5, 10, 20],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        }
    },
    'Neural Network': {
        'model': MLPClassifier(max_iter=500),
        'params': {
            'hidden_layer_sizes': [(50, 50), (100,), (100, 50)],
            'activation': ['tanh', 'relu'],
            'alpha': [0.0001, 0.001, 0.01]
        }
    }
}

# Fonction d'optimisation des hyperparamètres avec Optuna
def objective(trial, X_train, y_train):
    classifier_name = trial.suggest_categorical('classifier', list(models.keys()))
    classifier_info = models[classifier_name]
    classifier = classifier_info['model']
    params = classifier_info['params']

    trial_params = {}
    for param, values in params.items():
        if isinstance(values[0], (int, float)):
            trial_params[param] = trial.suggest_float(param, min(values), max(values))
        else:
            trial_params[param] = trial.suggest_categorical(param, values)
    
    classifier.set_params(**trial_params)
    
    pipeline = Pipeline(steps=[
        ('classifier', classifier)
    ])
    
    score = cross_val_score(pipeline, X_train, y_train, n_jobs=-1, cv=5, scoring='accuracy', error_score='raise')
    accuracy = score.mean()
    return accuracy

# Fonction pour ré-entraîner un modèle jusqu'à atteindre une amélioration significative
def retrain_model(best_pipeline, X_train, y_train, X_test, y_test, threshold=0.01, max_iter=10):
    previous_score = 0
    for iteration in range(max_iter):
        best_pipeline.fit(X_train, y_train)
        y_pred = best_pipeline.predict(X_test)
        current_score = accuracy_score(y_test, y_pred)
        improvement = current_score - previous_score
        if improvement < threshold:
            break
        previous_score = current_score
        print(f"Iteration {iteration + 1}, Accuracy: {current_score}, Improvement: {improvement}")
    return best_pipeline, current_score

# Fonction de visualisation
def plot_roc_curve(y_test, y_pred_proba, model_name, output_dir):
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba[:, 1])
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic - {model_name}')
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(output_dir, f'roc_curve_{model_name}.png'))
    plt.close()

def plot_confusion_matrix(y_test, y_pred, model_name, output_dir):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.savefig(os.path.join(output_dir, f'confusion_matrix_{model_name}.png'))
    plt.close()

def plot_cross_val_scores(model_scores, model_names, output_dir):
    plt.figure(figsize=(12, 8))
    sns.boxplot(x=model_names, y=model_scores)
    plt.xlabel('Model')
    plt.ylabel('Cross-Validation Score')
    plt.title('Model Comparison - Cross-Validation Scores')
    plt.xticks(rotation=45)
    plt.savefig(os.path.join(output_dir, 'cross_val_scores.png'))
    plt.close()

# Méthode principale
def train_and_evaluate_models(base_path, output_folder, target_columns, testing=False, chunk_size=1000, testing_sub_path_name='test'):
    all_scores = {}
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('application_train.csv'):
                file_path = os.path.join(root, file)
                test_file_path = file_path.replace('application_train.csv', 'application_test.csv')

                print(f"Processing file: {file_path}")

                # Lire les fichiers CSV
                train_data = pd.read_csv(file_path)
                test_data = pd.read_csv(test_file_path)

                for target_column in target_columns:
                    print(f"Using target column: {target_column}")

                    # Séparation des features et de la cible
                    X_train = train_data.drop(target_column, axis=1)
                    y_train = train_data[target_column]
                    X_test = test_data.drop(target_column, axis=1)
                    y_test = test_data[target_column]

                    # Optimisation des hyperparamètres avec Optuna
                    study = optuna.create_study(direction='maximize')
                    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100)

                    print('Number of finished trials: ', len(study.trials))
                    print('Best trial:')
                    trial = study.best_trial

                    print('  Value: ', trial.value)
                    print('  Params: ')
                    for key, value in trial.params.items():
                        print('    {}: {}'.format(key, value))

                    # Entraîner le meilleur modèle
                    best_classifier_name = trial.params['classifier']
                    best_classifier_info = models[best_classifier_name]
                    best_classifier = best_classifier_info['model']
                    best_params = {k: v for k, v in trial.params.items() if k != 'classifier'}

                    best_classifier.set_params(**best_params)

                    # Création du pipeline avec le meilleur modèle
                    best_pipeline = Pipeline(steps=[
                        ('classifier', best_classifier)
                    ])

                    best_pipeline, best_score = retrain_model(best_pipeline, X_train, y_train, X_test, y_test)

                    # Stocker les scores de validation croisée
                    all_scores[best_classifier_name] = study.trials_dataframe().value.values

                    # Déterminer le chemin de sortie
                    relative_path = os.path.relpath(root, base_path)
                    if testing:
                        output_dir = os.path.join(output_folder, testing_sub_path_name, relative_path, target_column)
                    else:
                        output_dir = os.path.join(output_folder, relative_path, target_column)
                        
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)

                    model_path = os.path.join(output_dir, f'best_{best_classifier_name}_model.pkl')
                    joblib.dump(best_pipeline, model_path)

                    # Évaluation du modèle
                    y_pred = best_pipeline.predict(X_test)
                    y_pred_proba = best_pipeline.predict_proba(X_test)
                    accuracy = accuracy_score(y_test, y_pred)
                    roc_auc = roc_auc_score(y_test, y_pred)

                    print(f"Accuracy: {accuracy}")
                    print(f"ROC AUC: {roc_auc}")
                    print(confusion_matrix(y_test, y_pred))
                    print(classification_report(y_test, y_pred))

                    # Logging avec mlflow
                    mlflow.set_experiment('credit_scoring')
                    with mlflow.start_run():
                        mlflow.log_params(trial.params)
                        mlflow.log_metric('accuracy', accuracy)
                        mlflow.log_metric('roc_auc', roc_auc)
                        mlflow.sklearn.log_model(best_pipeline, 'model')
                        mlflow.log_artifact(file_path)
                        mlflow.log_artifact(test_file_path)

                    print(f'Model saved at {model_path}')

                    # Visualisation des résultats
                    plot_roc_curve(y_test, y_pred_proba, best_classifier_name, output_dir)
                    plot_confusion_matrix(y_test, y_pred, best_classifier_name, output_dir)

    # Visualisation des scores de validation croisée
    model_names = list(all_scores.keys())
    model_scores = [score for scores in all_scores.values() for score in scores]
    model_names_repeated = [model for model in model_names for _ in range(len(all_scores[model]))]

    plot_cross_val_scores(model_scores, model_names_repeated, output_folder)





The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


NameError: name 'LogisticRegression' is not defined

## Method settings

In [ ]:
## Call

In [ ]:
train_and_evaluate_models(
    base_path=LOCAL_EXPORT_MANUAL_CHECK_PATCH_FOLDER_PATH, 
    output_folder=LOCAL_EXPORT_MODELIZATION_FOLDER_PATH, 
    target_columns=TARGET_COLUMNS, 
    testing=TESTING_MODE, 
    chunk_size=GENERAL_CHUNK_SIZE, 
    testing_sub_path_name=TESTING_MODE_SUB_FOLDER_NAME)